In [3]:
import numpy as np
from collections import defaultdict as dd
from random import choice
# import string
# from random import randint

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/My\ Drive/nlp

/content/drive/My Drive/nlp


In [6]:
file = open('supertags.txt', 'r')

tags = dict()
for i in file:
    e, f = i.split(' ')
    e = e.lower()
    if f[-1] == '\n':
        f = f[:-1]
    tags[e] = f

file.close()
def tag(s):
    if s in tags:
        return tags[s]
    else:
        return ('^' + s)[-3:]

In [7]:
file = open('pan-tadeusz.txt', 'r')
alfabet = 'filmujrzeźżądańpośćgnębchłystków'

def erase_nonlet(x):
    if x in alfabet:
        return x
    else:
        return ''

PT = list()
for e in file:
    if e[-1] == '\n':
        e = e[:-1]
    if e == '':
        continue
    e = e.lower().split(' ')
    good = True
    for i in range(len(e)):
        e[i] = ''.join(map(erase_nonlet, list(e[i])))
        if e[i] == '':
            continue
        if e[i] not in tags:
            good = False
            break
    f = []
    if not good:
        continue
    for x in e:
        if x == '':
            continue
        f.append(x)
#     print(f)
    PT.append(f)

file.close()

In [8]:
bigrams = dd(lambda : dd(int))
file = open('2grams.txt', 'r')

for e in file:
    if e[-1] == '\n':
        e = e[:-1]
    e = e.lower()
    l, first, second = e.split(' ')
    bigrams[first][second] += int(l)
    if int(l) <= 1:
        break

file.close()

file = open('unigrams.txt', 'r')
unigrams = dict()
for e in file:
    if e[-1] == '\n':
        e = e[:-1]
    e = e.lower()
    first, second = e.split()
    unigrams[first] = int(second)

file.close()

In [9]:
vowels = 'aeiouyóąę'
def zgloski(w):
    res = 0
    for i in range(len(w)):
        if w[i] == 'i' and i + 1 < len(w):
            if w[i+1] in vowels:
                continue
        if w[i] in vowels:
#             print(w[i])
            res +=1
    return res


In [10]:
def rymuj(s1, s2):
    s1 = list(s1)
    s2 = list(s2)
    s1.reverse()
    s2.reverse()
    v = 0
    for i in range(len(s1)):
        if len(s2) <= i:
            return False
        if s1[i] != s2[i]:
            return False
        if s1[i] == 'i' and i > 0:
            if s1[i-1] in vowels:
                continue
        if s1[i] in vowels:
            v += 1
        if v >= 2:
            break
    return True
            

In [11]:
korp = dd(set)
for e in unigrams:
    tg = tag(e)
    korp[tg].add((e, unigrams[e]))


korp_b = dd(set)
for e in bigrams:
    for f in bigrams[e]:
        tag2 = tag(f)
        korp_b[(e, tag2)].add((f, bigrams[e][f]))

In [12]:
def lines():
    num = np.random.randint(1, high=len(PT)-2)
    l1 = PT[num]
    tag_l1 = []
    sylab1 = []
    for e in l1:
        tag_l1.append(tags[e])
        sylab1.append(zgloski(e))
    if sum(sylab1) != 13:
        return lines()
    if rymuj(PT[num-1][-1], l1[-1]):
        l2 = PT[num-1]
    elif rymuj(PT[num+1][-1], l1[-1]):
        l2 = PT[num+1]
    else:
        return lines()
    tag_l2 = []
    sylab2 = []
    for e in l2:
        tag_l2.append(tags[e])
        sylab2.append(zgloski(e))
    if sum(sylab2) != 13:
        return lines()
    return l1, tag_l1, sylab1, l2, tag_l2, sylab2

In [13]:
sum_of_bigrams = 0
for e in bigrams:
    for f in bigrams[e]:
        sum_of_bigrams += bigrams[e][f]

sum_of_unigrams = sum(unigrams[e] for e in unigrams)

def P_big(w1, w2):
    return np.log(bigrams[w1][w2]/sum_of_bigrams + 1e-10)

def P_uni(w1):
    return np.log(unigrams[w1]/sum_of_unigrams + 1e-10)

def PMI(w1,w2):
    return P_big(w1, w2) - (P_uni(w1) + P_uni(w2))

def PPMI(w1, w2):
    return max(PMI(w1,w2), .0)

In [14]:
N=10
recall = 0

def backtracking_unigram(tag_list, zgl_list, rhymes=None):
    global recall
    if recall <= 0:
        return None
    tag = tag_list[0]
    zgl = zgl_list[0]
    backtracks = 0
    words = np.array([y[0] for y in filter(lambda x: zgloski(x[0]) == zgl, korp[tag])])
    probs = np.array([y[1] for y in filter(lambda x: zgloski(x[0]) == zgl, korp[tag])])
    probs = probs/np.sum(probs)
    if words.size == 0:
        recall -= 1
        return False
    res = np.random.choice(words, size=1, p=probs)[0]
    if len(tag_list) <= 1 and rhymes != None:
        while not rymuj(rhymes, res):
            res = np.random.choice(words, size=1, p=probs)[0]
            backtracks += 1
            recall -= 1
            if recall <= 0:
                return None
            if backtracks > N:
                recall -= 1
                return False
        return [res]
    while True:
        nexts = backtracking(res, tag_list[1:], zgl_list[1:], rhymes)
        if nexts == None:
            return None
        if nexts:
            return [res] + nexts
        res = np.random.choice(words, size=1, p=probs)[0]
        backtracks += 1
        if backtracks > N:
            recall -= 1
            return False

def backtracking(s, tag_list, zgl_list, rhymes=None):
    global recall
    if recall <= 0:
        return None
#     print(len(tag_list))
    tag = tag_list[0]
    zgl = zgl_list[0]
    if (s, tag) not in korp_b:
        recall -= 1
        return False
    backtracks = 0
    words = np.array([y[0] for y in filter(lambda x: zgloski(x[0]) == zgl, korp_b[(s, tag)])])
    probs = np.array([PPMI(s, y[0]) for y in filter(lambda x: zgloski(x[0]) == zgl, korp_b[(s, tag)])])
    probs = np.exp(probs)/np.sum(np.exp(probs))
    # print(probs)
    # probs = np.array([y[1] for y in filter(lambda x: zgloski(x[0]) == zgl, korp_b[(s, tag)])])
    # probs = probs/np.sum(probs)
    if words.size == 0:
        recall -= 1
        return False
    res = np.random.choice(words, size=1, p=probs)
#     print(len(tag_list))
    if len(tag_list) <= 1 and rhymes != None:
        while not rymuj(rhymes, res[0]):
            res = np.random.choice(words, size=1, p=probs)
            recall -= 1
            if recall <= 0:
                return None
            backtracks += 1
            if backtracks > N:
                recall -= 1
                return False
        return [res[0]]
    if len(tag_list) <= 1:
        return [res[0]]
    while True:
#         print(backtracks)
        nexts = backtracking(res[0], tag_list[1:], zgl_list[1:], rhymes) 
        if nexts == None:
            return None
        if not nexts:
            backtracks += 1
        else:
#             print(res[0], nexts)
            return [res[0]] + nexts
        if backtracks > N:
            return backtracking_unigram(tag_list, zgl_list, rhymes)
        res = np.random.choice(words, size=1, p=probs)



def poetry():
    global recall
    recall = 50
    # print('hej')
    _, l1_tag, l1_zgl, _, l2_tag, l2_zgl = lines()
#     _, l2_tag, l2_zgl = line()
    res1 = backtracking_unigram(l1_tag, l1_zgl)
    if res1:
        # print('ho')
        res2 = backtracking(res1[-1], l2_tag, l2_zgl, res1[-1])
        if res2:
            return (' '.join(res1), ' '.join(res2))
    return poetry()
    
    

In [ ]:
poetry()

('kult ten pozostał w części z świadomości ludu',
 'jak gdyby na prawnego zatwierdzenie cudu')

In [ ]:
print(poetry())

('bo bagno nie jest grzegorz nie schnie co to pany', 'i nie zda się pomocą odbierać w kajdany')


In [ ]:
print(poetry())

('jest na terenie nowy dziś turysta może', 'koncepcja go zostawi u nas co daj boże')


In [ ]:
print(poetry())

('pan radny z mikołajem kulą na dół drogą', 'i jeszcze się do woli dołożyć nie mogą')


In [ ]:
print(poetry())

('w swojej z taśm rolnych zwrócił auto od siebie', 'dając być przez rolnika na czerwonym żlebie')


In [ ]:
print(poetry())

('mam cel to wieś szlachecka to są wszystkie strony', 'mnie polityka grozi jeżeli z lizbony')


In [ ]:
print(poetry())

('nawet lekarza który już mógł bez potrzeby', 'oznacza służyć długo ma które potrzeby')


In [ ]:
print(poetry())

('pan jest z autobusem autosan z nami', 'wszyscy wolni nazwiska powstają ze łzami')


In [ ]:
lines()

(['albo', 'na', 'imieniny', 'pańskie', 'lub', 'na', 'łowy'],
 ['S7', 'S229', 'T3061', 'T2819', 'T1633', 'S229', 'T3514'],
 [2, 1, 4, 2, 1, 1, 2],
 ['sproszona', 'na', 'sejm', 'albo', 'sejmik', 'powiatowy'],
 ['T3798', 'S229', 'T3168', 'S7', 'T3168', 'T3038'],
 [3, 1, 1, 2, 2, 4])

In [ ]:
print(poetry())

('trudno jak na budowę trzeba jeszcze było', 'bowiem na stanowisku czy się przydarzyło')


In [ ]:
print(poetry())

('i wszyscy raz kolejny punktem pokonali', 'los społeczny przez polski i wszyscy zdołali')


In [ ]:
from tqdm import tqdm
for _ in tqdm(range(1000)):
    poetry()

100%|██████████| 1000/1000 [25:57<00:00,  1.56s/it]


In [ ]:
from tqdm import tqdm
for _ in tqdm(range(250)):
    poetry()

100%|██████████| 250/250 [18:09<00:00,  4.36s/it]


In [ ]:
print(poetry())

('na ruchem i wart blisko na pomoc dostają', 'a starzy i maszyny miasto otaczają')


In [ ]:
for i in range(15):
    print(poetry())

('a wprost tylko przejściową na lat trzy została', 'prawna względnie leczniczej szpital podjąć miała')
('i na podstawie równie jak i pod rozpadzie', 'i po błocie czy pęknie jak i po rozpadzie')
('prawników z bratem zająć na serią na steli', 'na czyjś paszport otrzymał producent modeli')
('dróg ten łysy mężczyzna owinięty w strawy', 'i planem położony od piątku od sprawy')
('jeśli car a gracz będzie bo klasy zajmują', 'a w punkcie wioski najpierw rosjanie się czują')
('na kartach przedstawiono kluczowe organy', 'na sposobem przepisy wdrożono w bociany')
('chciałbym mu wypowiedzieć stosunek udowy', 'głęboki do złożenia niniejszej umowy')
('dziennego punktów zdołał przed proces poboru', 'ochrona już praktycznie mówiąc się horroru')
('to państwa biuro pewne jak o nim gadałem', 'i miał kontakt podpisał marszałek z nawałem')
('tej serii swoje swoje sprawki i spawania', 'i sam i toż kapłaństwo i moje pytania')
('lecz w czyich bankach jeszcze za przyszłość wysłany', 'w rodzinnym ministerstwie 

In [ ]:
for i in range(15):
    print(poetry())

('jako wtedy się dobro dla kina kosztuje', 'normalnie nic tu użyć w łóżku nie lukruje')
('które dążyło w rynku ministra pytanie', 'w końcu wybrano grunty pod konkretne danie')
('wychowywała pana on już był za drzwiami', 'tak głucha sprawa siłą i bratanicami')
('czyli nie miał być pusty więc marszałek znowu', 'leżał się że gdzie młody psiak nie ma narowu')
('każda historia króla kapelana jana', 'pod poziomem komisja tak jest przepychana')
('iż ważniejszy dla skrajnych pań bardziej rozległy', 'i bladą cerę czy jest warszawa miał z cegły')
('kiedy nam szef nik przyjąć z ustawy uchwali', 'i dużo akt sporządzić na centralnej hali')
('jest po powrocie każdy dziś pracownik może', 'ustawa go wyzwoli u nas co daj boże')
('na rynkiem i grac nawet po pomoc czekają', 'a starzy i wycieczki biuro odwiedzają')
('miarę kandydaturę pana głosowałem', 'często choć tylko parę razy ją widziałem')
('dużo choć tylko parę razy ją widziałem', 'miarę emeryturę pana głosowałem')
('teraz jakoś ta żywa legenda poc